
# <b>Stored Video Service Demo</b>


The Vision Service now offers Stored Video Processing. This notebook aims to provide overall clarity about the Stored Video features to the user in terms of requirements and usage.
<ul>
    <li><font size="2">The output response file is stored at the object storage location specified in the below cells.</font></li>
    <li><font size="2">At the end of the notebook the response is downloaded and saved as <code>video_response.json</code> in the current directory. </font></li>
</ul>


### Steps to run the notebook:
- Notebook session setup
    - Installing the OCI Python SDK.
    - Setup sample input video.
    - Setup helper functions.
- Importing the required modules
- Setting the input variables
    - The user can give input variables of their choice.
- Running the main pipeline
    - Run all cells to get the output in the current directory. 

### Notebook session setup
<details>
    <summary>Instructions</summary>
    <ul>
        <li><font size="2">The user needs to setup only once.</font></li>
        <li><font size="2">Uncomment the commented cells and run once to setup.</font></li>
        <li><font size="2">Comment back the same cells to avoid running again.</font></li>
    </ul>
</details>

#### Installing the OCI Python SDK
Make sure to finish lab prerequisites and have access to SDK. 

In [ ]:
# Please use pip/pip3 as per availability
# !pip3 install --trusted-host=artifactory.oci.oraclecorp.com -i https://artifactory.oci.oraclecorp.com/api/pypi/global-dev-pypi/simple -U oci==2.115.2+preview.1.1678

#### Setup sample input

* Uncomment and run the cell below.
* User can create a bucket in their tenancy (user may skip this step if they have an existing bucket).
* Upload the video `demo.mp4` to the bucket (user can also upload and use the video of their choice).
* Input video limits and requirements
    - Input video can be of format ".mov", ".mp4", ".h264", ".mkv", or ".webm"
    - Maximum length of the input video is 1hr
    - Maximum size of the input video is 20 GB

In [ ]:
# !wget "https://raw.githubusercontent.com/oracle-samples/oci-data-science-ai-samples/main/labs/ai-vision/analyze_video_workshop/data/demo.mp4"
# !mkdir data
# !mv demo.mp4 data

#### Setup helper functions

In [ ]:
# !wget "https://raw.githubusercontent.com/oracle-samples/oci-data-science-ai-samples/main/labs/ai-vision/analyze_video_workshop/helper/analyze_video_utils.py"
# !mkdir helper
# !mv analyze_video_utils.py helper

#### Import Libraries

In [ ]:
import time
import oci
import json
from oci.ai_vision.models import *
from helper.analyze_video_utils import display_formatted_response
from IPython.display import JSON

#### Authorize OCI config
Set up authentication for OCI by reading configuration from a file. The default configuration file location is ```~/.oci/config``` and ```DEFAULT``` profile is chosen.

In [ ]:
config = oci.config.from_file('~/.oci/config', profile_name='DEFAULT')

#### Set required input variables
- **input_location_path:**
    The user needs to provide the following details:
    - <code>namespace</code> : specify the namespace where the input video is uploaded.
    - <code>bucket</code> : specify the bucket name where the input video is uploaded.
    - <code>filename</code> : specify the filename of the input video (e.g: we have uploaded <code>demo.mp4</code> in <code>Setup sample input</code> step).
- **output_location_path:**
    The user needs to provide the following details to store the output:
    - <code>namespace</code> : specify the namespace where the output has to be stored.
    - <code>bucket</code> : specify the bucket name where the output has to be stored.
    - <code>prefix</code> : specify the prefix where the output has to be stored.
- **compartment_id:**
    The user should provide the compartment OCID to call the API.

In [ ]:
input_namespace = "<INPUT_NAMESPACE>"
input_bucket = "<INPUT_BUCKET>"
input_filename = "<INPUT_FILENAME>"

output_namespace = "<OUTPUT_NAMESPACE>"
output_bucket = "<OUTPUT_BUCKET>"
output_prefix = "<OUTPUT_PREFIX>"

compartment_id = "<COMPARTMENT_ID>"

#### Set optional input variables
- **max_results:**
    Provide the maximum results needed for Object, Label and Face Detection. This is an upper limit over the output classes, the API may detect lesser classes according to the input video. Default value is 20.
- **min_confidence:**
    Provide the minimum confidence needed for the feature, between 0 and 1. This is an lower limit over the output, the API may detect objects and classes above the specified confidence. Default value is 0.35f. 
- **is_landmark_required:**
    Provide whether the landmark details for the face is required or not. Landmarks are the key points representing regions on the face. Default value is False.
- **model_id:**
    In case of custom models uncomment the line and provide the model OCID needed for Custom Label Detection and Object Detection features.

In [ ]:
max_results = 5

min_confidence = 0

is_landmark_required = False

# od_model_id = "<Object Detection MODEL_ID>"
# ld_model_id = "<Label Detection MODEL_ID>"

#### Setup input location

In [ ]:
object_location_1 = ObjectLocation()
object_location_1.namespace_name = input_namespace
object_location_1.bucket_name = input_bucket
object_location_1.object_name = input_filename
object_locations = [object_location_1]
input_location = ObjectListInlineInputLocation()
input_location.object_locations = object_locations

#### Setup output location

In [ ]:
output_location = OutputLocation()
output_location.namespace_name = output_namespace
output_location.bucket_name = output_bucket
output_location.prefix = output_prefix

### Create AIServiceVisionClient and Setup input features for Stored Video Processing
User can specify the features they want to call. In the below code all supported pretrained models are called. To call custom models uncomment commented lines and replace corresponding model id for the feature.

In [ ]:
ai_service_vision_client = oci.ai_vision.AIServiceVisionClient(config=config)

video_label_detection_feature = VideoLabelDetectionFeature()
video_label_detection_feature.max_results = max_results
video_label_detection_feature.min_confidence = min_confidence
# video_label_detection_feature.model_id = ld_model_id

video_object_detection_feature = VideoObjectDetectionFeature()
video_object_detection_feature.max_results = max_results
video_object_detection_feature.min_confidence = min_confidence
# video_object_detection_feature.model_id = od_model_id

video_text_detection_feature = VideoTextDetectionFeature()
video_text_detection_feature.min_confidence = min_confidence

video_face_detection_feature = VideoFaceDetectionFeature()
video_face_detection_feature.max_results = max_results
video_face_detection_feature.min_confidence = min_confidence
video_face_detection_feature.is_landmark_required = is_landmark_required

features = [video_label_detection_feature, 
            video_object_detection_feature, 
            video_text_detection_feature, 
            video_face_detection_feature]

### Create video job

In [ ]:
create_video_job_details = CreateVideoJobDetails()
create_video_job_details.features = features
create_video_job_details.compartment_id = compartment_id
create_video_job_details.output_location = output_location
create_video_job_details.input_location = input_location

res = ai_service_vision_client.create_video_job(create_video_job_details=create_video_job_details)


### Job submitted
The job is created and should be in <code>ACCEPTED</code> state.

In [ ]:
JSON(display_formatted_response(res))

#### Job IN_PROGRESS
The job progress is tracked till completion with a polling interval of 5 seconds.

In [ ]:
job_id = res.data.id
polling_interval = 5
print("Job ID :", job_id, '\n')
seconds = 0
while res.data.lifecycle_state == "IN_PROGRESS" or res.data.lifecycle_state == "ACCEPTED":
    print(f"Job {job_id} is IN_PROGRESS for {str(seconds)} seconds, progress: {res.data.percent_complete}")
    time.sleep(polling_interval)
    seconds += polling_interval
    res = ai_service_vision_client.get_video_job(video_job_id=job_id)

### Job completed
The job is completed and should be in <code>SUCCEEDED</code> state. In case the job is not in <code>SUCCEEDED</code> state refer to the displayed response below to know the state and reason. Possible job states are <code>PARTIALLY SUCCEEDED</code> and <code>FAILED</code>

In [ ]:
JSON(display_formatted_response(res))

#### Get response json from object storage
The output can be found in the output location specified or it can be saved in ```video_response.json``` file by running the below cell.

In [ ]:
object_storage_client = oci.object_storage.ObjectStorageClient(config)
output_object_name = f"{output_location.prefix}/{job_id}/{input_location.object_locations[0].object_name}.json"

video_response = object_storage_client.get_object(output_location.namespace_name, output_location.bucket_name, output_object_name)

response = json.dumps(video_response.data.text, indent=4)
file = open('video_response.json', 'w')
file.write(response)